In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('train.csv') 
y_train=train_data.SalePrice


collist = train_data.columns.tolist()
# print(collist)
collist.remove('SalePrice')
X_train = train_data[collist]

# train_data.describe
train_data_encoded=pd.get_dummies(X_train)

train_X, test_X, train_y, test_y = train_test_split(train_data_encoded.as_matrix(), y_train.as_matrix(), test_size=0.1)
train_X=pd.DataFrame(train_X)
test_X=pd.DataFrame(test_X)
final_train, final_test = train_X.align(test_X, join='left',axis=1)

my_imputer = Imputer()
train_data_encoded_imputed = my_imputer.fit_transform(final_train)
test_data_encoded_imputed = my_imputer.transform(final_test)

model = XGBRegressor()

model.fit(train_data_encoded_imputed, train_y, early_stopping_rounds=5, 
             eval_set=[(test_data_encoded_imputed, test_y)],verbose=False)
preds_test = model.predict(test_data_encoded_imputed)
preds_train = model.predict(train_data_encoded_imputed)
MAE_train=mean_absolute_error(np.log(train_y),np.log(preds_train))
MAE_test=mean_absolute_error(np.log(test_y),np.log(preds_test))
print('preds train', np.mean(preds_train))
print('y train',np.mean(train_y))
print('preds test',np.mean(preds_test))
print('MAE log train',MAE_train)
print('MAE log test',MAE_test)


C:\Users\Administrateur.000\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


preds train 181303.77
y train 181370.13926940638
preds test 178215.6
MAE log train 0.06837031184989077
MAE log test 0.08395652708823247


In [19]:
data = pd.read_csv('test.csv') 
X=data[collist]
data_encoded=pd.get_dummies(X)
final_train, final_data = train_X.align(data_encoded, join='left',axis=1)
data_encoded_imputed = my_imputer.transform(final_data)
preds=model.predict(data_encoded_imputed)
print('preds test',np.mean(preds))

preds test 170889.25


In [16]:
my_submission = pd.DataFrame({'Id': data.Id, 'SalePrice': preds})
my_submission.to_csv('submission 3.csv', index=False)